In [1]:
import pygame 
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix
import itertools
import cv2
import matplotlib.image as mpimg
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

file=r'../neironka/train.csv'
fileTest=r'../neironka/test.csv'
train=pd.read_csv(file)
import math
f, ax = plt.subplots(5, 5)
for i in range(1,26):
    data = train.iloc[i,1:785].values 
    nrows, ncols = 28, 28
    grid = data.reshape((nrows, ncols))
    n=math.ceil(i/5)-1
    m=[0,1,2,3,4]*5
    ax[m[i-1], n].imshow(grid)
    
dataTest=pd.read_csv(fileTest)
trainNumbers=train['label']
train=train.drop(labels=['label'],axis = 1)
trainNumbers= to_categorical(trainNumbers,num_classes=10)

train=train/255
test=dataTest/255
train=train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

model = Sequential()
model.add(Conv2D(24, (3, 3), padding='same', input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,  
        rotation_range=5, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  

datagen.fit(train)
model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])
model.fit(train, trainNumbers, epochs=5, batch_size=120)

    
def make_predict():
    img = mpimg.imread('predict.png')[..., 1]
    img = cv2.resize(img, dsize=(28,28))
    img = np.array(img).reshape(1,28,28,1)
    pred = model.predict(img)
    print(pred)
    return np.argmax(pred,axis=1)[0]
    
    
    
pygame.init()
scr=pygame.display.set_mode((600,400))
scr.fill((0,0,0))
pygame.display.update()
txt=pygame.font.Font(None,30)
clock=pygame.time.Clock()
FPS=60

p=True
checker=False
while p:
    for i in pygame.event.get():
        if i.type==pygame.QUIT:
            p=False
        if i.type==pygame.MOUSEBUTTONDOWN:
            if (i.button==1):
                checker=True
            if(i.button==3):
                pygame.image.save(scr,'predict.png')
                pr=pygame.image.load('predict.png')
                prediction = make_predict()
                text=txt.render('Prediction:{0}'.format(prediction),True, (255,255,255))
                scr.blit(text,(16,16))
                pygame.display.update()
        if i.type==pygame.KEYDOWN:
            if i.key==pygame.K_r:
                scr.fill((0,0,0))
        if i.type==pygame.MOUSEBUTTONUP:
            if (i.button==1):
                checker=False
        if i.type==pygame.MOUSEMOTION and checker==True:
            pygame.draw.circle(scr, (255,255,255), i.pos, 10)

        
        
                
                
    clock.tick(FPS)
    pygame.display.update()



pygame 2.0.0 (SDL 2.0.12, python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Epoch 1/5
350/350 [==============================] - 9s 25ms/step - loss: 0.2497 - accuracy: 0.9268
Epoch 2/5
350/350 [==============================] - 9s 26ms/step - loss: 0.0823 - accuracy: 0.9756
Epoch 3/5
350/350 [==============================] - 9s 25ms/step - loss: 0.0495 - accuracy: 0.9856
Epoch 4/5
350/350 [==============================] - 9s 24ms/step - loss: 0.0342 - accuracy: 0.9897
Epoch 5/5
350/350 [==============================] - 9s 25ms/step - loss: 0.0219 - accuracy: 0.9940
[[1.1218402e-04 4.1121066e-01 8.6575061e-02 1.3630153e-01 3.9853943e-03
  2.5697377e-01 2.8977622e-03 2.2514457e-02 1.5248974e-03 7.7904195e-02]]
[[0.1996402  0.312383   0.02993187 0.00137547 0.16674122 0.09275333
  0.10867061 0.07723073 0.00594626 0.00532734]]
[[8.2573266e-04 3.9185152e-06 6.8967737e-04 6.7704096e-03 1.3059494e-04
  3.1367663e-02 9.4100446e-01 1.3992473e-02 1.9456538e